In [2]:
!pip install tabtransformertf
!pip install tensorflow-addons
from IPython.display import clear_output
clear_output()

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
import tensorflow_addons as tfa
from tabtransformertf.utils.preprocessing import df_to_dataset, build_categorical_prep
from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rema-final-dataset/REMA_Full_Data.csv


In [4]:
df = pd.read_csv('/kaggle/input/rema-final-dataset/REMA_Full_Data.csv')
df = df.sample(frac=1).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)
df


,bedroom,price_clean,Neighborhood,listing,type,building_age_years,furnishing,land_area,number_of_floors,City,floor_lower,surface_area,bathroom,floor,floor_numeric
0,0.0,120000.0,344,sale,3,20.0,0.0,937.0,2.0,8,0,2.0,0.0,0.0,0.000000
1,5.0,240.0,123,rent,apartment,7.5,0.0,NaN,1.0,4,second floor,200.0,3.0,second floor,2.000000
2,3.0,125000.0,428,sale,0,0.5,0.5,0.0,1.0,2,first floor,160.0,2.0,1.0,1.000000
3,4.0,14000.0,4,rent,apartment,3.0,0.0,NaN,1.0,2,ground floor,270.0,5.0,ground floor,0.000000
4,3.0,24000.0,10,rent,apartment,3.0,1.0,NaN,1.0,2,first floor,180.0,3.0,first floor,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11647,4.0,90000.0,71,sale,0,7.5,0.0,0.0,1.0,2,second floor,189.0,4.0,2.0,2.000000
11648,5.0,100080.0,355,sale,1,7.5,0.0,710.0,2.0,2,ground floor,300.0,3.0,0.0,0.000000
11649,0.5,90000.0,240,sale,1,7.5,0.0,1000.0,3.0,13,ground floor,220.0,3.0,0.0,0.000000
11650,0.5,250.0,249,rent,apartment,7.5,1.0,NaN,1.0,2,second floor,52.0,1.0,second floor,2.000000


In [5]:
#final preprocessing steps to prepare and get the dataset ready
type_dict = {
    "apartment":0,
    "villas and palaces":2,
    "farms and chalets":4
}
df["type"] = df["type"].replace(type_dict).astype(int)

In [6]:
#df['land_area'] = df['land_area'].fillna(df['land_area'].median())

df.drop(columns=['land_area','floor_lower','floor'], inplace=True)
#dataset shuffling twice

In [7]:
num_cols = ['bedroom', 'building_age_years', 'number_of_floors', 
            'surface_area', 'bathroom', 'floor_numeric', 'furnishing']
cat_cols = ['Neighborhood', 'listing', 'type', 'City']

target_col = 'price_clean'

df[target_col] = np.log1p(df[target_col])

In [8]:
for col in cat_cols:
    df[col] = df[col].astype(str)

In [13]:
X = df[num_cols + cat_cols]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling: Fit on TRAIN, transform both
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [14]:
train_df_full = pd.concat([X_train, y_train], axis=1)
test_df_full = pd.concat([X_test, y_test], axis=1)

# create TF model two datasets train and test sets
# the model expects an input of type dictionary as df_to_dataset clearly provides
train_dataset = df_to_dataset(train_df_full, target_col, shuffle=True, batch_size=256)
test_dataset = df_to_dataset(test_df_full, target_col, shuffle=False, batch_size=256)

# building the categorical prep layers
category_prep_layers = build_categorical_prep(train_df_full, cat_cols)


/opt/conda/lib/python3.7/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
100%|██████████| 4/4 [00:00<00:00, 189.20it/s]


In [15]:
ft_encoder = FTTransformerEncoder(
    numerical_features=num_cols,
    categorical_features=cat_cols,
    numerical_data=X_train[num_cols].values,   # Use X_train stats
    categorical_data=X_train[cat_cols].values, # Use X_train vocab
    y=None,
    numerical_embedding_type='linear',
    embedding_dim=32,
    depth=8,
    heads=10,
    attn_dropout=0.2,  # Slightly lower dropout
    ff_dropout=0.2,
    explainable=True
)

# building the FT-Transformer model
ft_model = FTTransformer(
    encoder=ft_encoder,
    out_dim=1,
    out_activation=None  # OPTIMIZATION: Linear output is safer for regression
)

# compiling + preparing AdamW optimzer
optimizer = tfa.optimizers.AdamW(
    learning_rate=0.0001, 
    weight_decay=0.000001
)

ft_model.compile(
    optimizer=optimizer,
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)


In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

print("\n##########  Training Initiated...  ##########")
history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)

# --------------------------------
# 6. Predict & Evaluate
# --------------------------------
print("\n Generating predictions...")
preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

# OPTIMIZATION: Inverse Log Transform to get real prices
pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n############################################")
print("\n######## { FT-Transformer Results } ########")
print("\n############################################\n")
print("MAE ==>", mean_absolute_error(y_test_real, pred_real))
print("RMSE ==>", np.sqrt(mean_squared_error(y_test_real, pred_real)))
print("R2 ==>", r2_score(y_test_real, pred_real))
ft_model.save("REMA_ft_transformer_model")


##########  Training Initiated...  ##########
Epoch 1/100
37/37 [==============================] - 13s 86ms/step - loss: 135.2164 - importances_loss: 99.0488 - output_loss: 36.1676 - importances_root_mean_squared_error: 9.9523 - output_root_mean_squared_error: 6.0139 - val_loss: 113.1940 - val_importances_loss: 98.9721 - val_output_loss: 14.2219 - val_importances_root_mean_squared_error: 9.9485 - val_output_root_mean_squared_error: 3.7712
Epoch 2/100
37/37 [==============================] - 2s 49ms/step - loss: 113.5150 - importances_loss: 99.0488 - output_loss: 14.4662 - importances_root_mean_squared_error: 9.9523 - output_root_mean_squared_error: 3.8034 - val_loss: 110.4696 - val_importances_loss: 98.9721 - val_output_loss: 11.4975 - val_importances_root_mean_squared_error: 9.9485 - val_output_root_mean_squared_error: 3.3908
Epoch 3/100
37/37 [==============================] - 2s 48ms/step - loss: 110.9407 - importances_loss: 99.0488 - output_loss: 11.8919 - importances_root_mean_sq

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

print("\n##########  Training Initiated...  ##########")
history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)

print("\n Generating predictions...")
preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

# OPTIMIZATION step: taking the {Inverse Log} to Transform back data to get real listings prices
pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n############################################")
print("\n######## { FT-Transformer Results } ########")
print("\n############################################\n")
print("MAE ==>", mean_absolute_error(y_test_real, pred_real))
print("RMSE ==>", np.sqrt(mean_squared_error(y_test_real, pred_real)))
print("R2 ==>", r2_score(y_test_real, pred_real))
ft_model.save("REMA_ft_transformer_model")


##########  Training Initiated...  ##########
Epoch 1/100
37/37 [==============================] - 22s 91ms/step - loss: 154.9096 - importances_loss: 99.3672 - output_loss: 55.5424 - importances_root_mean_squared_error: 9.9683 - output_root_mean_squared_error: 7.4527 - val_loss: 124.1460 - val_importances_loss: 97.6990 - val_output_loss: 26.4470 - val_importances_root_mean_squared_error: 9.8843 - val_output_root_mean_squared_error: 5.1427
Epoch 2/100
37/37 [==============================] - 2s 49ms/step - loss: 125.2259 - importances_loss: 99.3672 - output_loss: 25.8587 - importances_root_mean_squared_error: 9.9683 - output_root_mean_squared_error: 5.0851 - val_loss: 117.5561 - val_importances_loss: 97.6990 - val_output_loss: 19.8571 - val_importances_root_mean_squared_error: 9.8843 - val_output_root_mean_squared_error: 4.4561
Epoch 3/100
37/37 [==============================] - 2s 49ms/step - loss: 120.3057 - importances_loss: 99.3672 - output_loss: 20.9385 - importances_root_mean_sq

In [135]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

print("\n##########  Training Initiated...  ##########")
history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)


print("\n Generating predictions...")
preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n############################################")
print("\n######## { FT-Transformer Results } ########")
print("\n############################################\n")
print("MAE ==>", mean_absolute_error(y_test_real, pred_real))
print("RMSE ==>", np.sqrt(mean_squared_error(y_test_real, pred_real)))
print("R2 ==>", r2_score(y_test_real, pred_real))
ft_model.save("REMA_ft_transformer_model")


##########  Training Initiated...  ##########
Epoch 1/100
37/37 [==============================] - 14s 86ms/step - loss: 143.9810 - importances_loss: 98.8121 - output_loss: 45.1688 - importances_root_mean_squared_error: 9.9404 - output_root_mean_squared_error: 6.7208 - val_loss: 115.0159 - val_importances_loss: 99.9185 - val_output_loss: 15.0974 - val_importances_root_mean_squared_error: 9.9959 - val_output_root_mean_squared_error: 3.8855
Epoch 2/100
37/37 [==============================] - 2s 47ms/step - loss: 113.6177 - importances_loss: 98.8121 - output_loss: 14.8055 - importances_root_mean_squared_error: 9.9404 - output_root_mean_squared_error: 3.8478 - val_loss: 110.4874 - val_importances_loss: 99.9185 - val_output_loss: 10.5689 - val_importances_root_mean_squared_error: 9.9959 - val_output_root_mean_squared_error: 3.2510
Epoch 3/100
37/37 [==============================] - 2s 48ms/step - loss: 110.0490 - importances_loss: 98.8121 - output_loss: 11.2368 - importances_root_mean_sq

In [54]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

print("\n##########  Training Initiated...  ##########")
history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)


print("\n Generating predictions...")
preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n############################################")
print("\n######## { FT-Transformer Results } ########")
print("\n############################################\n")
print("MAE ==>", mean_absolute_error(y_test_real, pred_real))
print("RMSE ==>", np.sqrt(mean_squared_error(y_test_real, pred_real)))
print("R2 ==>", r2_score(y_test_real, pred_real))
ft_model.save("REMA_ft_transformer_model")


##########  Training Initiated...  ##########
Epoch 1/100
37/37 [==============================] - 12s 79ms/step - loss: 135.6713 - importances_loss: 98.7872 - output_loss: 36.8840 - importances_root_mean_squared_error: 9.9392 - output_root_mean_squared_error: 6.0732 - val_loss: 108.3514 - val_importances_loss: 100.0180 - val_output_loss: 8.3334 - val_importances_root_mean_squared_error: 10.0009 - val_output_root_mean_squared_error: 2.8868
Epoch 2/100
37/37 [==============================] - 2s 46ms/step - loss: 107.1738 - importances_loss: 98.7872 - output_loss: 8.3866 - importances_root_mean_squared_error: 9.9392 - output_root_mean_squared_error: 2.8960 - val_loss: 106.7563 - val_importances_loss: 100.0180 - val_output_loss: 6.7382 - val_importances_root_mean_squared_error: 10.0009 - val_output_root_mean_squared_error: 2.5958
Epoch 3/100
37/37 [==============================] - 2s 46ms/step - loss: 105.3778 - importances_loss: 98.7872 - output_loss: 6.5905 - importances_root_mean_sq

In [108]:
###### ORIGINAL PREDICTION CODE ######
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

print("\n##########  Training Initiated...  ##########")
history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)

print("\n Generating predictions...")
preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n############################################")
print("\n######## { FT-Transformer Results } ########")
print("\n############################################\n")
print("MAE ==>", mean_absolute_error(y_test_real, pred_real))
print("RMSE ==>", np.sqrt(mean_squared_error(y_test_real, pred_real)))
print("R2 ==>", r2_score(y_test_real, pred_real))
ft_model.save("REMA_ft_transformer_model")


##########  Training Initiated...  ##########
Epoch 1/100
37/37 [==============================] - 6s 43ms/step - loss: 122.7078 - importances_loss: 99.4905 - output_loss: 23.2173 - importances_root_mean_squared_error: 9.9745 - output_root_mean_squared_error: 4.8184 - val_loss: 104.4795 - val_importances_loss: 97.8211 - val_output_loss: 6.6584 - val_importances_root_mean_squared_error: 9.8905 - val_output_root_mean_squared_error: 2.5804
Epoch 2/100
37/37 [==============================] - 1s 21ms/step - loss: 103.5241 - importances_loss: 99.4905 - output_loss: 4.0336 - importances_root_mean_squared_error: 9.9745 - output_root_mean_squared_error: 2.0084 - val_loss: 99.2905 - val_importances_loss: 97.8211 - val_output_loss: 1.4694 - val_importances_root_mean_squared_error: 9.8905 - val_output_root_mean_squared_error: 1.2122
Epoch 3/100
37/37 [==============================] - 1s 21ms/step - loss: 100.7743 - importances_loss: 99.4906 - output_loss: 1.2837 - importances_root_mean_squared_

In [13]:
####### ORINIGAL MODEL RESULTS #######

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

print("\n##########  Training Initiated...  ##########")
history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)


print("\n Generating predictions...")
preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n############################################")
print("\n######## { FT-Transformer Results } ########")
print("\n############################################\n")
print("MAE ==>", mean_absolute_error(y_test_real, pred_real))
print("RMSE ==>", np.sqrt(mean_squared_error(y_test_real, pred_real)))
print("R2 ==>", r2_score(y_test_real, pred_real))


##########  Training Initiated...  ##########
Epoch 1/100
37/37 [==============================] - 1s 24ms/step - loss: 100.1219 - importances_loss: 99.4907 - output_loss: 0.6312 - importances_root_mean_squared_error: 9.9745 - output_root_mean_squared_error: 0.7945 - val_loss: 98.6621 - val_importances_loss: 97.8212 - val_output_loss: 0.8409 - val_importances_root_mean_squared_error: 9.8905 - val_output_root_mean_squared_error: 0.9170
Epoch 2/100
37/37 [==============================] - 1s 21ms/step - loss: 100.1318 - importances_loss: 99.4907 - output_loss: 0.6411 - importances_root_mean_squared_error: 9.9745 - output_root_mean_squared_error: 0.8007 - val_loss: 98.6153 - val_importances_loss: 97.8212 - val_output_loss: 0.7940 - val_importances_root_mean_squared_error: 9.8905 - val_output_root_mean_squared_error: 0.8911
Epoch 3/100
37/37 [==============================] - 1s 21ms/step - loss: 100.0986 - importances_loss: 99.4908 - output_loss: 0.6079 - importances_root_mean_squared_er

In [11]:
### SECOND TRY {TF-TRANSFORMER MODEL} ###

# --------------------------------
# 1. Load Data & Identify Features
# --------------------------------
df = pd.read_csv('/kaggle/input/rema-final-dataset/REMA_Full_Data.csv')

# Handle Missing Values
df['land_area'] = df['land_area'].fillna(df['land_area'].median())

# Define Columns
num_cols = ['bedroom', 'building_age_years', 'land_area', 'number_of_floors', 
            'surface_area', 'bathroom', 'floor_numeric', 'furnishing']
cat_cols = ['Neighborhood', 'listing', 'type', 'City', 'floor_lower']
target_col = 'price_clean'

# OPTIMIZATION: Log-transform the target variable to fix skewness/negative R2
# We will inverse this later using np.expm1
df[target_col] = np.log1p(df[target_col])

# Ensure categorical columns are strings
for col in cat_cols:
    df[col] = df[col].astype(str)

# --------------------------------
# 2. Split Data (X_train, X_test)
# --------------------------------
# Split into Train and Test
X = df[num_cols + cat_cols]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling: Fit on TRAIN, transform both
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# --------------------------------
# 3. Prepare TF Datasets
# --------------------------------
# Recombine for df_to_dataset utility
train_df_full = pd.concat([X_train, y_train], axis=1)
test_df_full = pd.concat([X_test, y_test], axis=1)

# Create TF Datasets
# The model expects a dictionary inputs, which df_to_dataset provides
train_dataset = df_to_dataset(train_df_full, target_col, shuffle=True, batch_size=256)
test_dataset = df_to_dataset(test_df_full, target_col, shuffle=False, batch_size=256)

# Build categorical prep layers
category_prep_layers = build_categorical_prep(train_df_full, cat_cols)

# --------------------------------
# 4. FT-Transformer Encoder
# --------------------------------
ft_encoder = FTTransformerEncoder(
    numerical_features=num_cols,
    categorical_features=cat_cols,
    numerical_data=X_train[num_cols].values,   # Use X_train stats
    categorical_data=X_train[cat_cols].values, # Use X_train vocab
    y=None,
    numerical_embedding_type='linear',
    embedding_dim=32,
    depth=3,
    heads=6,
    attn_dropout=0.2,  # Slightly lower dropout
    ff_dropout=0.2,
    explainable=True
)

ft_model = FTTransformer(
    encoder=ft_encoder,
    out_dim=1,
    out_activation=None  # OPTIMIZATION: Linear output is safer for regression
)


optimizer = tfa.optimizers.AdamW(
    learning_rate=0.001, 
    weight_decay=0.0001
)

ft_model.compile(
    optimizer=optimizer,
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10, 
    restore_best_weights=True
)

history = ft_model.fit(
    train_dataset,
    epochs=100, 
    validation_data=test_dataset,
    callbacks=[early_stopping],
    verbose=1
)

preds = ft_model.predict(test_dataset)
pred_log = preds['output'].flatten()

# optimization step
pred_real = np.expm1(pred_log)
y_test_real = np.expm1(y_test.values)

print("\n REGRESSION MODEL ==> { FT-Transformer }")
print("MAE:", mean_absolute_error(y_test_r, pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test_r, pred)))
print("R2 Score:", r2_score(y_test_r, pred))

/opt/conda/lib/python3.7/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
100%|██████████| 5/5 [00:00<00:00, 192.22it/s]

Starting training...
Epoch 1/100


37/37 [==============================] - 6s 42ms/step - loss: 139.7571 - importances_loss: 99.5963 - output_loss: 40.1608 - importances_root_mean_squared_error: 9.9798 - output_root_mean_squared_error: 6.3373 - val_loss: 117.4747 - val_importances_loss: 97.9258 - val_output_loss: 19.5490 - val_importances_root_mean_squared_error: 9.8957 - val_output_root_mean_squared_error: 4.4214
Epoch 2/100
37/37 [==============================] - 1s 23ms/step - loss: 113.7522 - importances_loss: 99.5963 - output_loss: 14.1559 - importances_root_mean_squared_error: 9.9798 - output_root_mean_squared_error: 3.7624 - val_loss: 106.9879 - val_importances_loss: 97.9258 - val_output_loss: 9.0622 - val_importances_root_mean_squared_error: 9.8957 - val_output_root_mean_squared_error: 3.0103
Epoch 3/100
37/37 [==============================] - 1s 22ms/step - loss: 107.2878 - importances_loss: 99.5963 - output_loss: 7.6915 - importances_root_mean_squared_error: 9.9798 - output_root_mean_squared_error: 2.7734 -